🎬 1. Extract Audio from Video

In [1]:
%pip install moviepy speechrecognition pydub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from moviepy.editor import VideoFileClip

video = VideoFileClip("Subtitle-Video/video/Demo-2.mp4")
video.audio.write_audiofile("Subtitle-Video/temp_audio/temp_audio.wav")

MoviePy - Writing audio in Subtitle-Video/temp_audio/temp_audio.wav


MoviePy - Done.


🧠 2. Transcribe Audio to Text (Speech Recognition)

In [3]:
%pip install haystack-ai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from haystack.components.audio import LocalWhisperTranscriber
from pydub import AudioSegment
from pydub.silence import split_on_silence

In [5]:
# Initialize the Whisper transcriber
transcriber = LocalWhisperTranscriber()
transcriber.warm_up()

In [6]:
transcription = transcriber.run(sources=["./Subtitle-Video/temp_audio/temp_audio.mp3"])

In [7]:
print(transcription["documents"][0].content)

 Sir Keir Starmer has hit the ground running on his first full day as Prime Minister, holding his first Cabinet meeting and then a press conference laying out his plans for the immediate future. He'll start a tour of the UK's nations tomorrow and warned tough decisions will be made early. He said both prisons and the NHS in England are broken, but that work to fix the NHS has already begun. And he confirmed the previous government's Rwanda migrants plan has been scrapped. Here's our political correspondent, Shihab Khan, on Sir Keir's first full day in charge.


In [8]:
transcript_text = transcription["documents"][0].content

In [9]:
transcript_text

" Sir Keir Starmer has hit the ground running on his first full day as Prime Minister, holding his first Cabinet meeting and then a press conference laying out his plans for the immediate future. He'll start a tour of the UK's nations tomorrow and warned tough decisions will be made early. He said both prisons and the NHS in England are broken, but that work to fix the NHS has already begun. And he confirmed the previous government's Rwanda migrants plan has been scrapped. Here's our political correspondent, Shihab Khan, on Sir Keir's first full day in charge."

In [10]:
with open("./Subtitle-Video/transcript/transcript-en.txt", "w") as file:
    file.write(transcript_text)

In [11]:
import os

In [12]:
sound = AudioSegment.from_wav("Subtitle-Video/temp_audio/temp_audio.wav")

In [13]:
chunks = split_on_silence(
    sound,
    min_silence_len=300,                # Detect shorter pauses (was 700)
    silence_thresh=sound.dBFS - 16,     # Be more sensitive to soft speech
    keep_silence=300                    # Add 300ms silence before/after each chunk
)

In [14]:
subtitles = []
start_time = 0

In [15]:
from pydub.silence import detect_nonsilent

nonsilent_ranges = detect_nonsilent(
    sound,
    min_silence_len=500,
    silence_thresh=sound.dBFS - 16
)

# Extend each detected range by 300ms (within bounds)
buffer_ms = 300
nonsilent_ranges = [
    (max(start - buffer_ms, 0), min(end + buffer_ms, len(sound)))
    for start, end in nonsilent_ranges
]

In [16]:
for i, (start_ms, end_ms) in enumerate(nonsilent_ranges):
    chunk = sound[start_ms:end_ms]
    chunk_filename = os.path.abspath(f"Subtitle-Video/temp_audio/chunk{i}.wav")
    chunk.export(chunk_filename, format="wav")

    # Load the .wav file and convert to .mp3
    wav_audio = AudioSegment.from_wav(chunk_filename)
    chunk_mp3_path = os.path.abspath(f"Subtitle-Video/temp_audio/chunk{i}.mp3")
    wav_audio.export(chunk_mp3_path, format="mp3")

In [17]:
%pip install translate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from translate import Translator

In [19]:
translator= Translator(to_lang="fr") # alternate between Chinese (zh) and French text (fr)

In [20]:
import re

def clean_transcription(text):
    """Remove English name translations while keeping abbreviations."""
    # Regular expression to remove English names inside parentheses, but not abbreviations
    cleaned_text = re.sub(r'（ [A-Za-z\s]+ ）', '', text)
    return cleaned_text

In [21]:
translated_text = ""

for i, (start_ms, end_ms) in enumerate(nonsilent_ranges):
    print(f"Start : {start_ms}, type {type(start_ms)}")
    print(f"End: {end_ms}, type: {type(end_ms)}")
    chunk = sound[start_ms:end_ms]
    chunk_filename = f"./Subtitle-Video/temp_audio/chunk{i}.mp3"
    chunk.export(chunk_filename, format="mp3")

    try:
        # Use the absolute path to avoid errors
        transcription_result = transcriber.transcribe(sources=[chunk_filename])
        text = transcription_result[0].content
        text = translator.translate(text)
        # text = clean_transcription(text) 
        # for Chinese characters. 
        translated_text += text

        if text:
            print(f"Transcribed text: {text}")
            start_time = start_ms / 1000.0
            end_time = end_ms / 1000.0
            subtitles.append((start_time, end_time, text))
    except Exception as e:
        print(f"Error during transcription: {e}")
        continue

    # Remove the temporary audio file
    os.remove(chunk_filename)

Start : 0, type <class 'int'>
End: 10683, type: <class 'int'>
Transcribed text: Sir Keir Starmer a démarré sa première journée complète en tant que Premier ministre, tenant sa première réunion du Cabinet, puis une conférence de presse exposant ses plans pour l'avenir immédiat.
Start : 10628, type <class 'int'>
End: 23337, type: <class 'int'>
Transcribed text: Il commencera une tournée des pays du Royaume-Uni demain et a averti que des décisions difficiles seraient prises tôt. Il a dit que les deux prisons et le NHS en Angleterre sont en panne, mais que le travail pour réparer le NHS a déjà commencé.
Start : 23321, type <class 'int'>
End: 28383, type: <class 'int'>
Transcribed text: et il a confirmé que le plan du gouvernement précédent pour les migrants rwandais a été abandonné.
Start : 28548, type <class 'int'>
End: 33733, type: <class 'int'>
Transcribed text: Voici notre correspondant politique, Shihab Khan, lors de la première journée complète de Sir Keir.


In [22]:
translated_text

"Sir Keir Starmer a démarré sa première journée complète en tant que Premier ministre, tenant sa première réunion du Cabinet, puis une conférence de presse exposant ses plans pour l'avenir immédiat.Il commencera une tournée des pays du Royaume-Uni demain et a averti que des décisions difficiles seraient prises tôt. Il a dit que les deux prisons et le NHS en Angleterre sont en panne, mais que le travail pour réparer le NHS a déjà commencé.et il a confirmé que le plan du gouvernement précédent pour les migrants rwandais a été abandonné.Voici notre correspondant politique, Shihab Khan, lors de la première journée complète de Sir Keir."

In [24]:
# writing the french version. 
with open("./Subtitle-Video/transcript/transcript-fr.txt", "w", encoding="utf-8") as file:
    file.write(translated_text)

In [ ]:
# writing the chinese version. 
with open("./Subtitle-Video/transcript/transcript-zh.txt", "w", encoding="utf-8") as file:
    file.write(translated_text)

💬 3. Burn Subtitles into Video

In [ ]:
from moviepy.editor import TextClip, CompositeVideoClip

In [ ]:
# import moviepy.config as mpy_config
# mpy_config.change_settings({"IMAGEMAGICK_BINARY": r"C:\Program Files\ImageMagick-7.1.1-Q16-HDRI\convert.exe"})

In [ ]:
from moviepy.config import change_settings
change_settings({"IMAGEMAGICK_BINARY": r"C:\\Program Files\\ImageMagick-7.1.1-Q16-HDRI\\magick.exe"})

In [ ]:
from moviepy.editor import TextClip
# solution to MoviePy not found error: https://stackoverflow.com/questions/51928807/moviepy-cant-detect-imagemagick-binary-on-windows

clip = TextClip("Hello, world!", fontsize=70, color='white', bg_color='black')
clip.save_frame("test_output.png")

In [ ]:
import textwrap

In [ ]:
# Split text for Chinese language
def split_text(text, max_chars=8):
    """Split Chinese text into meaningful chunks based on punctuation and length."""
    # Split the text based on punctuation marks, including commas
    chunks = re.split(r'([。！？，\n])', text)
    result = []
    current_chunk = ""

    for chunk in chunks:
        # Ignore empty strings and punctuation-only chunks
        if not chunk or re.fullmatch(r'[。！？，\n]', chunk):
            continue
        
        # If adding the chunk exceeds max length, start a new chunk
        if len(current_chunk) + len(chunk) > max_chars:
            if current_chunk:
                result.append(current_chunk)
            current_chunk = chunk
        else:
            current_chunk += chunk
    
    # Append the last chunk if not empty
    if current_chunk:
        result.append(current_chunk)

    return result

In [ ]:
# Example usage
text = """凯尔·斯塔默爵士在担任总理的第一全天开始运作，举行了他的第一次内阁会议，然后举行了新闻发布会，阐述了他对近期的计划。他将于明天开始对英国国家的巡回演出，并警告说将提前做出艰难的决定。他说，英格兰的监狱和NHS都被打破了，但修复NHS的工作已经开始。他证实，前政府的卢旺达移民计划已经被取消。这是我们的政治记者谢哈布·汗在基尔爵士执政的第一天。"""
print("Text chunks:", split_text(text))

In [ ]:
# import re

# def split_text(text, max_words=8):
#     """Split a sentence into smaller chunks with up to `max_words` each."""
#     words = text.split()
#     return [' '.join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

In [ ]:
subtitle_clips = []

chinese_font = "SimHei"  # You can replace this with another font on your system

for start, end, text in subtitles:
    duration = end - start
    # chunks = split_text(text, max_words=8)
    chunks = split_text(text)

    print(f"Text chunk: {chunks}")

    word_counts = [len(chunk.split()) for chunk in chunks]
    total_words = sum(word_counts)

    # Compute proportional durations for each chunk
    chunk_durations = [(wc / total_words) * duration for wc in word_counts]

    current_time = start
    for chunk, chunk_duration in zip(chunks, chunk_durations):
        chunk_end = current_time + chunk_duration

        wrapped_text = textwrap.fill(chunk, width=50)

        txt_clip = TextClip(
            wrapped_text,
            fontsize=64,
            color='white',
            font=chinese_font,
            method='caption',
            size=(int(video.w * 0.9), None)
        )

        txt_clip = txt_clip.on_color(
            size=txt_clip.size,
            color=(0, 0, 0),
            col_opacity=0.6
        ).set_position(("center", "bottom"))

        txt_clip = txt_clip.set_start(current_time).set_duration(chunk_duration)
        subtitle_clips.append(txt_clip)

        current_time = chunk_end

In [ ]:
final_video = CompositeVideoClip([video] + subtitle_clips, size=video.size)
final_video.write_videofile("Subtitle-Video/output/Output-Demo2.mp4", codec="libx264", fps=video.fps)